In [2]:
! pip3 install pandas

     |████████████████████████████████| 11.5 MB 76 kB/s  eta 0:00:01
     |████████████████████████████████| 15.7 MB 724 kB/s eta 0:00:01
     |████████████████████████████████| 503 kB 2.6 MB/s eta 0:00:01


In [3]:
import pandas as pd

In [1]:
# import findspark

# findspark.init()

In [4]:
import pyspark

from pyspark.sql import SparkSession

In [6]:
spark=SparkSession.builder.master('spark://DESKTOP-HSK5ETQ.localdomain:7077').appName('Step1-Word-Processing').getOrCreate()

### Word Count

In [7]:
file_path = "./data/news.txt"
text_file = spark.sparkContext.textFile(file_path)

def clean_punc(x):
  punc='!"”“#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-0123456789۰۱۲۳۴۵۶۷۸۹'
  for ch in punc:
    x = x.replace(ch, '')
  x=x.replace('‌','')
  return x.strip()

text_file = text_file.map(clean_punc)
words = text_file.flatMap(lambda line: line.split(" "))
words = words.filter(lambda x:x!='')
wordCounts_rdd = words.map(lambda  word:('TotalWords',1))
wordCounts = wordCounts_rdd.reduceByKey(lambda a,b:a+b).collect()
wordCounts


21/12/19 16:49:27 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/12/19 16:49:42 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/12/19 16:49:57 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/12/19 16:50:12 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/12/19 16:50:27 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/12/19 16:50:42 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

[('TotalWords', 23151)]

In [8]:
words.take(10)

['فاطمی',
 'امین',
 'را',
 'بیشتر',
 'با',
 'ستاد',
 'تبصره',
 'میشناسند،',
 'ستادی',
 'که']

In [10]:
word_pairs = words.map(lambda  word:(word,1))
print(word_pairs.take(10))
wordCounts = word_pairs.reduceByKey(lambda a,b:a+b).sortBy(lambda a: -a[1])
results=wordCounts.take(20)
results

[('فاطمی', 1), ('امین', 1), ('را', 1), ('بیشتر', 1), ('با', 1), ('ستاد', 1), ('تبصره', 1), ('میشناسند،', 1), ('ستادی', 1), ('که', 1)]


[('و', 1035),
 ('در', 933),
 ('به', 618),
 ('از', 572),
 ('این', 524),
 ('که', 510),
 ('است', 392),
 ('را', 359),
 ('با', 307),
 ('برای', 168),
 ('کشور', 136),
 ('بر', 112),
 ('آن', 109),
 ('یک', 105),
 ('دولت', 97),
 ('خود', 95),
 ('شده', 94),
 ('هم', 82),
 ('قرار', 81),
 ('تا', 78)]

In [11]:
stop_words=wordCounts.map(lambda w : w[0]).take(20)
stop_words

['و',
 'در',
 'به',
 'از',
 'این',
 'که',
 'است',
 'را',
 'با',
 'برای',
 'کشور',
 'بر',
 'آن',
 'یک',
 'دولت',
 'خود',
 'شده',
 'هم',
 'قرار',
 'تا']

#### Bigrams

In [13]:
line="خدایا چنان کن سرانجام کار که تو خوشنود باشی و ما رستگار"
print(line)
words=[w for w in line.split(" ") if w not in stop_words and len(w.strip())>1]
print(words[:-1])
print(words[1:])
print("-"*50)
bigrams= [(w1,w2) for w1,w2 in zip(words[:-1],words[1:])]
bigrams

خدایا چنان کن سرانجام کار که تو خوشنود باشی و ما رستگار
['خدایا', 'چنان', 'کن', 'سرانجام', 'کار', 'تو', 'خوشنود', 'باشی', 'ما']
['چنان', 'کن', 'سرانجام', 'کار', 'تو', 'خوشنود', 'باشی', 'ما', 'رستگار']
--------------------------------------------------


[('خدایا', 'چنان'),
 ('چنان', 'کن'),
 ('کن', 'سرانجام'),
 ('سرانجام', 'کار'),
 ('کار', 'تو'),
 ('تو', 'خوشنود'),
 ('خوشنود', 'باشی'),
 ('باشی', 'ما'),
 ('ما', 'رستگار')]

In [14]:
def get_bigrams(line):
    words=[w for w in line.split(" ") if w not in stop_words and len(w.strip())>1]
    bigrams= [(w1,w2) for w1,w2 in zip(words[:-1],words[1:])]
    return bigrams

In [15]:
bigrams = text_file.flatMap(get_bigrams)\
            .map(lambda bigram : (bigram,1)) \
            .reduceByKey(lambda a,b : a+b) \
            .filter(lambda x : x[1] > 5)\
            .sortBy(lambda x : -x[1]) 
            

bigrams.collect()


[(('بورس', 'کالا'), 26),
 (('قیمتهای', 'پایه'), 21),
 (('نشان', 'میدهد'), 19),
 (('فاطمی', 'امین'), 18),
 (('سال', 'گذشته'), 17),
 (('خواهد', 'بود'), 15),
 (('بازار', 'سیمان'), 14),
 (('داشته', 'باشد'), 13),
 (('تامین', 'مالی'), 13),
 (('حکمرانی', 'آب'), 12),
 (('وزرای', 'پیشنهادی'), 12),
 (('میلیارد', 'دلار'), 12),
 (('خواهد', 'شد'), 11),
 (('وجود', 'دارد'), 11),
 (('وزیر', 'صمت'), 10),
 (('ایران', 'خودرو'), 10),
 (('نظر', 'میرسد'), 9),
 (('مدیریت', 'آب'), 9),
 (('سیستم', 'حکمرانی'), 9),
 (('رای', 'اعتماد'), 9),
 (('رشد', 'اقتصادی'), 9),
 (('تحت', 'تاثیر'), 8),
 (('خودروهای', 'فرسوده'), 8),
 (('وزارت', 'صنعت،'), 8),
 (('صنعت،', 'معدن'), 8),
 (('معدن', 'تجارت'), 8),
 (('واردات', 'خودرو'), 8),
 (('وجود', 'ندارد'), 8),
 (('تامین', 'انرژی'), 7),
 (('داشته', 'باشند'), 7),
 (('امنیت', 'آبی'), 7),
 (('مجلس', 'شورای'), 7),
 (('قیمت', 'پایه'), 7),
 (('رضایت', 'مشتریان'), 6),
 (('همین', 'دلیل'), 6),
 (('عرضه', 'تقاضا'), 6),
 (('هزار', 'تن'), 6),
 (('صمت', 'سیزدهم'), 6),
 (('چهار', 'ماه'), 6),
 

In [16]:
df = pd.DataFrame(bigrams.collect())
df.columns=['Bigram','Count']
df

,Bigram,Count
0,"(بورس, کالا)",26
1,"(قیمتهای, پایه)",21
2,"(نشان, میدهد)",19
3,"(فاطمی, امین)",18
4,"(سال, گذشته)",17
5,"(خواهد, بود)",15
6,"(بازار, سیمان)",14
7,"(داشته, باشد)",13
8,"(تامین, مالی)",13
9,"(حکمرانی, آب)",12


In [17]:
spark.stop()